In [1]:
import pandas as pd

讀入交通資料

In [2]:
traffic_A1 = pd.read_csv("data/108A1.csv")
traffic_A2_First = pd.read_csv("data/108A2_first_half.csv")
traffic_A2_Second = pd.read_csv("data/108A2_second_half.csv")

讀入雨量資料

In [7]:
# w_All = pd.read_csv("data/W_Kaohsiung.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'data/W_Kaohsiung.csv'

將2019年所有資料放入

In [9]:
def put_event_in_area(district):
    output = []
    for i in range(0,len(traffic_A1["發生時間"])):
        if traffic_A1["發生地點"][i][0:3] == district:
            output.append([traffic_A1["發生時間"][i],traffic_A1["發生地點"][i],traffic_A1["死亡受傷人數"][i],traffic_A1["車種"][i],traffic_A1["經度"][i],traffic_A1["緯度"][i],"A1"])
    for i in range(0,len(traffic_A2_First["發生時間"])):
        if traffic_A2_First["發生地點"][i][0:3] == district:
            output.append([traffic_A2_First["發生時間"][i],traffic_A2_First["發生地點"][i],traffic_A2_First["死亡受傷人數"][i],traffic_A2_First["車種"][i],traffic_A2_First["經度"][i],traffic_A2_First["緯度"][i],"A2"])
    for i in range(0,len(traffic_A2_Second["發生時間"])):
        if traffic_A2_Second["發生地點"][i][0:3] == district:
            output.append([traffic_A2_Second["發生時間"][i],traffic_A2_Second["發生地點"][i],traffic_A2_Second["死亡受傷人數"][i],traffic_A2_Second["車種"][i],traffic_A2_Second["經度"][i],traffic_A2_Second["緯度"][i],"A2"])
    return output

In [10]:
Kaohsiung = put_event_in_area("高雄市")
NewTaipei = put_event_in_area("新北市")
Taichung = put_event_in_area("臺中市")
Tainan = put_event_in_area("臺南市")
Taipei = put_event_in_area("臺北市")
Taoyuan = put_event_in_area("桃園市")
all_data = []
for i in [Kaohsiung,NewTaipei,Taichung,Tainan,Taipei,Taoyuan]:
    all_data.extend(i)
    print(len(all_data))

43061
79843
122975
162287
185333
228427


讀入測站資料

In [11]:
print(all_data[0])

['108年01月01日 01時28分00秒', '高雄市岡山區成功路台上351路燈號前0.0公尺前', '死亡1;受傷0', '普通重型-機車;自用-小貨車(含客、貨兩用)', 120.301278, 22.814121, 'A1']


In [12]:
ob_station = pd.read_csv("data/observsation_station_fix.csv")

In [13]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):# 經度1，緯度1，經度2，緯度2 （十進制度數）
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
 
    # haversine
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 
    return c * r

In [14]:
print(haversine(121.442017,24.997647,120.301278,22.814121))

269.06333652610084


In [15]:
def get_distance(district):
    for i in range(0,len(district)):
        distance = {}
        for j in range(0,len(ob_station)):
            distance[ob_station["站名"][j]] = haversine(ob_station["經度"][j],ob_station["緯度"][j],district[i][4],district[i][5])
        district[i].append(sorted(distance.items(), key=lambda item: item[1])[0])
    return district

In [16]:
for i in all_data:
    i = get_distance([i])
print(all_data[0])

['108年01月01日 01時28分00秒', '高雄市岡山區成功路台上351路燈號前0.0公尺前', '死亡1;受傷0', '普通重型-機車;自用-小貨車(含客、貨兩用)', 120.301278, 22.814121, 'A1', ('岡山', 2.0027603920763686)]


In [135]:
# district = [Kaohsiung, NewTaipei, Taichung, Tainan, Taipei, Taoyuan]
# for i in district:
#     i = get_distance(i)
# print(district[0][0])

In [136]:
cols = ["發生時間","發生地點","死亡受傷人數","車種","經度","緯度","種類","測站"]

pd.DataFrame(Taichung, columns=cols).to_csv('data/updated/all_fix.csv')

In [137]:
# Kaohsiung = pd.read_csv("data/updated/Kaohsiung.csv")
# NewTaipei =  pd.read_csv("data/updated/NewTaipei.csv")
# Taichung =  pd.read_csv("data/updated/Taichung.csv")
# Tainan = pd.read_csv("data/updated/Tainan.csv")
# Taipei =  pd.read_csv("data/updated/Taipei.csv")
# Taoyuan = pd.read_csv("data/updated/Taoyuan.csv")

In [138]:
# ob_Kaohsiung = Kaohsiung["測站"].to_list()
# print(len(Kaohsiung["測站"]))
# print(ob_Kaohsiung)

In [139]:
# all_observation = []
# for j in district:
#     for i in j:
#         try:
#             all_observation.append(i[6][0])
#         except:
#             print(i)
# print(set(all_observation))
# print(len(set(all_observation)))

In [140]:
# all_observation_districk = []
# with open("a_b.txt","w") as file:
#     for i in all_observation:
#         num = ob_station.index[ob_station["站名"] == i].to_list()[0]
#         all_observation_districk.append([ob_station[num:num+1]["縣市"].to_list()[0],i])
#         file.write(str(ob_station[num:num+1]["縣市"].to_list()[0]))
#         file.write(str(i))
#         file.write("\n")
#     print(all_observation_districk)
#     print(len(all_observation_districk))

In [141]:
43132+ 185294

228426

In [142]:
118+143

261